Questions:
* Is 'roc_auc_score' an appropriate metric? What else could I use? I mreged previous_application with application_train, I got a higher 'roc_auc_score' but a lower submission score.
* A: 
* When handling categorical features. Do I need to use pd.factorize? see  https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772. I'm just using get_dummies() directly.
* A: You can use both, you have to what works best for your dataset. But both can work

DONE:
* For previous_application after I run the simple imputer I don't have any NaN in the np ndarray, but when I turn it into a pandas DataFrame it has 11109336. I need to turn it into a pandas DataFrame to merge it with application_train. What can I do?
* A: Question is not clear
* When using simple imputer, I lose the column names. It turns the pd.DataFrame into a np.ndarray. Is this the right process, or are there other ways to handle missing data, NaN, Inf, numbers too big for INT32, etc?
* A: Consider 'pd.fillna()'. https://pandas.pydata.org/pandas-docs/stable/missing_data.html
* Should I do one-hot encoding before or after merging the different datasets (ex, application_train & previous_application)?
* A: Merger, one-hot encoding fitting the model.
* I get a better competition score by training my model only with application_train.csv than what I get when I train it will ALL the datasets. Am I merging them correctly?
* A: Now that I do one-hot encoding BEFORE merging I get better results with ALL the data than with app_train alone.
* Creating Flat Dataset. When I mreged previous_application with application_train using .groupby().mean() it dropped the categorical features! How do I avoid this?
* A: Do one-hot encoding first.

June 7 through June 17
Future steps - Must do:
* Data visualization - See screenshot for variables to go after first: EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_ANNUITY, AMT_CREDIT, DAYS_ID_PUBLISH, pcb_CNT_INSTALMENT_FUTURE, DAYS_REGISTRATION, DAYS_EMPLOYED.
* Add feature importance graph for RFR
* Clean Notebook.
* Review Capstone project proposal response.
* Normalizing data - See screenshot for variables to go after first: EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_ANNUITY, AMT_CREDIT, DAYS_ID_PUBLISH, pcb_CNT_INSTALMENT_FUTURE, DAYS_REGISTRATION, DAYS_EMPLOYED.
* Handling skewed data - See screenshot for variables to go after first: EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_ANNUITY, AMT_CREDIT, DAYS_ID_PUBLISH, pcb_CNT_INSTALMENT_FUTURE, DAYS_REGISTRATION, DAYS_EMPLOYED.
* When running GridSearchCV then use Train Test Split, but when just training the model don't split the data. Is the competition test higher? Is this what the other kernels do?
* LGBM - Tune hyperparameters with GridSearchCV
* Review other kernels:
  - 0.789, feature engineering, LGBM: https://www.kaggle.com/shep312/lightgbm-harder-better-slower
  - 0.785, feature engineering, LGBM: https://www.kaggle.com/cklwankaggle/lightgbm-try
  - 0.782, feature engineering, LGBM: https://www.kaggle.com/gregory711/good-fun-with-ligthgbm
  - 0.781, feature engineering, LGBM, metric, kfold: https://www.kaggle.com/kosovanolexandr/ligthgbm-0-781-home-credit-default-risk
  - 0.779, EDA, feature engieering, handling skewed & missing data: https://www.kaggle.com/sukhyun9673/basic-baseline-with-lgb-v3-categocial-not-yet
  - 0.778, simple, LGBM: https://www.kaggle.com/ashukr/good-fun-with-ligthgbm
  - 0.777 feat eng, handle missing data?, LGBM, Kfold: https://www.kaggle.com/blackbee2016/good-fun-with-automation
  - ++ EDA, multiple regressors: https://www.kaggle.com/pavanraj159/loan-repayers-v-s-loan-defaulters-home-credit
  - +Mem reduction, hyperparameter tunning: https://www.kaggle.com/mlisovyi/lightgbm-hyperparameter-optimisation-lb-0-761
  - Flatenning dataset, LGBM hyperparameters: https://www.kaggle.com/shep312/lightgbm-with-weighted-averages-dropout-787/code
  - feature engineering, XGB model: https://www.kaggle.com/shenba/home-credit-eda-23may2018
  - R, EDA, XGB & LGBM models: https://www.kaggle.com/ambarish/eda-xgb-lgbm-home-credit-default-risk
  - EDA, LGBM: https://www.kaggle.com/kosovanolexandr/home-credit-default-risk-competition
  - EDA, no model: https://www.kaggle.com/gpreda/home-credit-default-risk-extensive-eda
  - -EDA: https://www.kaggle.com/gauravtaneja/home-credit-take0
* Try other regressors: GBRT, AdaBoost, SGDC
  - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor
  - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
* Try using other metrics to evaluate the model's performance to see if they more accureately align with the project score.
* Add reference to other kernels in mine:
  - https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772
  - https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm/code
Report:
* Preliminary results

June 18 through July 1
- Report:
* MAKE AN ENSEMBLE METHOD OF MULTIPLE MODELS
  - http://scikit-learn.org/stable/modules/ensemble.html#forest
* WEEK1: I should have a skeleton of all the areas of the report to share with mentor.
* WEEK2: I should have a first full version of the report to share with mentor and ready to submit

- Future steps - Nice to do:
* Only apply one-hot encoding to columns which are not numeric:
  - https://www.kaggle.com/dansbecker/using-categorical-data-with-one-hot-encoding
* Consider doing 'pd.factorize' for one-hot encoding
* CREATE A scikit-learn Pipeline
* Measuring model/learner performance using other evaluation metric (LR+, f1, r2)?
  - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
  - http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix
* Hyperparameter tunening (different values until I find a max, use higher percentage of training & validation data)
* Review Handling missing or invalid data, other better methods?
  - https://www.kaggle.com/dansbecker/handling-missing-values
* See if sklearn.model_selection.'KFold' could yield a better result than 'train_test_split'
* Consider other ways to merge data or new features for feature engineering:
  - https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm/code

DONE:
* Review Capstone project report requirements.
* Re-train LGBM model with same parameters but ALL data. Submit. 0.772. Same as the other kernel https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772
* Try one-hot encoding on the individual datasets before merging. Create a function. Do fillna after merging. Using the other data files (Feature Engineering, Create flat dataset), categorical features are being dropped.
* Re-train LGBM model with same parameters but app_train data only. Submit. 0.747.
* Re-train LGBM model with same parameters but ALL data. Submit. 0.745.
* Re-train model with same parameters but all data. Submit. 0.719.
* Handle missing and categorical variables for the Flat (ALL) dataset before splitting the data. Explain why. Submit. 0.732.
* Remove SK_ID_CURR from dataset before fitting the model
* Add feature importance - For LGBM & RFR (without graph)
* Try other regressor: LGBM - First pass complete, performance is much lower than example
* Using the other data files (Feature Engineering, Create flat dataset) first pass complete
* Handling missing or invalid data
* Split train & validation set
* Hyperparameter tunening
* Measuring model/learner performance using evaluation metric (auc_roc_score)
* Handle missing data or NaN, try with pd.fillna()

    Preliminary results:
- Categorical values turned into numerical features with one-hot encoding scheme
- Fill missing or wrong values
- RFR Default values, only application training data, score: 0.591
- RFR Best values with GridSearchCV, only application training data, score: 0.722 <- 0.62 (surpassing the 0.688 benchmark for Random Forrest)
- Got a score of 0.732 with the same model parameters, but by merging train & test datasets before doing pre-processing.
- RFR Best values with GridSearchCV, ALL available data, score: 0.709 & 0.710
- I'm still don't why with ALL the datasets I get a lower competition score than with just application_train.
- I'm still not sure why I keep getting a higher roc_auc_score with ALL the datasets than with just application_train, but the competition score is lower.
- LGBM Best values with GridSearchCV, ALL available data, score: 0.745
- LGBM Best values with GridSearchCV, app_train available data, score: 0.745
- I don't know why the score was so low. Based on the original kernel (https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772) the score should have been 0.772.
- Lessons learnt so far:
- Need and how to create a flat datasat from multiple datafiles. Explain what I did. Do some little research. Other ways to merge the data that doesn't use mean.
- Feature engineering. More art that science. Explain concept. They only new feature I created is the count of number of records when there are multiple number of records of some typer per unique load id SK_ID_CURR. What other features could be created? How? (from the result of the data analysis & experience)
- Problem when merging datasets. Categorical values were being dropped. when merging. Solution. Do one-hot encoding on each dataset before merging. Retest on RFR, LGBM with app_train only (shouldn't be needed, I already know whit score) with ALL the NEW dataset, it should have several new features from the merged datasets.
- Merge train and test datasets together to do the pre-processing. Explain. Makes the mean more meaningful when completing empty values with the column mean. And also makes sure the test dataset has the same features than the train set. Otherwise if the test set is smaller there is a chance that it doesn't have all the categorical features, so when you apply one-hot encoding you'll end up missing features and the prediction will fail because the model and the test dataset sizes don't match.
- ???? Best values with GridSearchCV, ALL available data, score: ? (PENDING)
- Handling missing data (Why don't they do it?)
- Handling Categorical variables, why do they use pd.factorize?
- Explain why I keep dataset merged before separating train and validation set. The mean used is more relevant, given that the test data set with be fewer samples. The reason to do one-hot encoding all together is because both the training and validation dataset need to have the same features, and doing it togethers ensures this.
- Normalizing Data (PENDING)
- Unskewing Data (PENDING)

Abbreviations:
RFR: Random Forest Regressor
????: Another regressor

# **Sections:**
[1. Import libraries & support functions](#import)  
[2. Dataset preparation](#data_prep)  
[3. Exploratory Data Analysis (EDA)](#eda)  
&nbsp; [3.1 Application Train](#eda_app_train)  
[4. Feature Engineering](#feat_eng)  
&nbsp; [4.1 Create a Flat Dataset](#flat_dataset)  
&nbsp; [4.2 Handle Skewed Continuous Data](#4.1)  
&nbsp; [4.3 Normalize Continuous Data](#4.2)  
&nbsp; [4.4 Handle Categorical Variables](#4.3)  
&nbsp; [4.5 Handle Missing Data](#4.4)  
&nbsp;&nbsp; [4.5.1 Previous Applications](#4.5)  
[5. Split Data into Training and Validation](#5)  
[6. Hyperparameter Tuning](#6)  
[7. Model Fitting & Prediction](#7)  

Acknowledgements:
- Flattening dataset, LGBM model starting point: https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772
- General ideas: https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm/code
- Reducing memory footprint: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

# <a id="import">1 Import Libraries and create support functions</a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSVfile I/O (e.g. pd.read_csv)
import os
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
from plotly import tools
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
from sklearn.ensemble import RandomForestRegressor
# http://lightgbm.readthedocs.io/en/latest/Python-Intro.html
# https://github.com/Microsoft/LightGBM
import lightgbm as lgb
# Add evaluation metric to measure the model's performance
# Regression metrics available:
# http://scikit-learn.org/stable/modules/classes.html#regression-metrics
# http://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
# http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
# Cannot use sklearn.metrics.accuracy_score as it is a Classification metric
from sklearn.metrics import make_scorer, r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split

#warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

In [2]:
# Enable Debugging while I test things
debugging = False # True or False

In [3]:
# Support functions
def bar_hor(df, col, title, color, w=None, h=None, lm=0, limit=100, return_trace=False, rev=False, xlb = False):
    cnt_srs = df[col].value_counts()
    yy = cnt_srs.head(limit).index[::-1] 
    xx = cnt_srs.head(limit).values[::-1] 
    if rev:
        yy = cnt_srs.tail(limit).index[::-1] 
        xx = cnt_srs.tail(limit).values[::-1] 
    if xlb:
        trace = go.Bar(y=xlb, x=xx, orientation = 'h', marker=dict(color=color))
    else:
        trace = go.Bar(y=yy, x=xx, orientation = 'h', marker=dict(color=color))
    if return_trace:
        return trace 
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

def bar_hor_noagg(x, y, title, color, w=None, h=None, lm=0, limit=100, rt=False):
    trace = go.Bar(y=x, x=y, orientation = 'h', marker=dict(color=color))
    if rt:
        return trace
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


def bar_ver_noagg(x, y, title, color, w=None, h=None, lm=0, rt = False):
    trace = go.Bar(y=y, x=x, marker=dict(color=color))
    if rt:
        return trace
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    
def gp(col, title):
    df1 = app_train[app_train["TARGET"] == 1]
    df0 = app_train[app_train["TARGET"] == 0]
    a1 = df1[col].value_counts()
    b1 = df0[col].value_counts()
    
    total = dict(app_train[col].value_counts())
    x0 = a1.index
    x1 = b1.index
    
    y0 = [float(x)*100 / total[x0[i]] for i,x in enumerate(a1.values)]
    y1 = [float(x)*100 / total[x1[i]] for i,x in enumerate(b1.values)]

    trace1 = go.Bar(x=a1.index, y=y0, name='Target : 1', marker=dict(color="#96D38C"))
    trace2 = go.Bar(x=b1.index, y=y1, name='Target : 0', marker=dict(color="#FEBFB3"))
    return trace1, trace2 

In [4]:
# This implementation was copied from: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# <a id="data_prep">2 Dataset preparation</a>

In [5]:
# List available data files
#print(os.listdir("../input"))
print("Loading data files...")

start = time()
# Load the Point of Sale Cash balance dataset
posc_bal = reduce_mem_usage(pd.read_csv("../input/POS_CASH_balance.csv"))
# Load the Bureau Balance dataset
bureau_bal = reduce_mem_usage(pd.read_csv("../input/bureau_balance.csv"))
# Load the Application Training dataset
app_train = reduce_mem_usage(pd.read_csv("../input/application_train.csv"))
# Load the Previous Applications dataset
prev_app = reduce_mem_usage(pd.read_csv("../input/previous_application.csv"))
# Load the Installements Payments dataset
inst_pay = reduce_mem_usage(pd.read_csv("../input/installments_payments.csv"))
# Load the Credit Card Balance dataset
cc_bal = reduce_mem_usage(pd.read_csv("../input/credit_card_balance.csv"))
# Load the Application Testing dataset
app_test = reduce_mem_usage(pd.read_csv("../input/application_test.csv"))
# Load the Bureau dataset
bureau = reduce_mem_usage(pd.read_csv("../input/bureau.csv"))
end = time()

print("Finished loading data files and running memory optimization in {} seconds.".format(int(round(end - start))))

# <a id="eda">3 Exploratory Data Analysis (EDA)</a>

## <a id="eda_app_train">3.1 Application Train dataset</a>

In [5]:
'''
# Display the first 5 records of application_train.csv
display(data_train.head())
# Display the first 5 records of application_test.csv
display(data_test.head())
# See the first 5 rows of the dataframe
display(prev_app.head())

# DataFrame statistics summary for selected columns
data_train[["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE"]].describe()
#print(len(data_train.columns))
'''

In [6]:
'''
# Total number of records
print("Total number of records in the train dataset: {:,}".format(len(data_train)))
# Total number of features. Excluding the load ids (SK_ID_CURR) and the target variable (TARGET).
print("Total number of features in the train dataset: {}".format(data_train.shape[1] - 2))

# Total number of records
print("Total number of records in the test dataset: {:,}".format(len(data_test)))
# Total number of features. Excluding the load ids (SK_ID_CURR). There is NO target variable (TARGET) in the test dataset.
print("Total number of features in the test dataset: {}".format(data_test.shape[1] - 1))

# Total number of features. Excluding the current loan ids (SK_ID_CURR) and the previous loan ids (SK_ID_PREV)
print("Total number of features of previous applications, before one-hot encoding: {}".format(prev_app.shape[1] - 2))
print("Previous applications has {:,} samples".format(prev_app.shape[0]))
# Check if therer is any NaN value in the dataset
print("Total number of NaN in the dataframe: {:,}".format(prev_app.isnull().sum().sum())) # TODO: REMOVE
'''

### <a id="eda_app_train_target">3.1.1 Target Label</a>

In [7]:
bar_hor(app_train, "TARGET", "Distribution of Target Variable" , ["#44ff54", '#ff4444'], h=350, w=600, lm=200, xlb = ['Target : 1','Target : 0'])

COMPLETE - Analysis of Target Variable
Definition: Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)

### <a id="eda_app_train_ext_source_1">3.1.? EXT_SOURCE_1</a>

In [8]:
#app_train['EXT_SOURCE_1'].isnull().sum()

In [9]:
app_train['EXT_SOURCE_1'].isnull().sum()
# There are many missing values
# WILL HAVE TO COME BACK TO THIS.
# BUT GIVEN IT IS ALREADY NORMALIZED, THERE IS NOT MUCH I CAN CHANGE
#plt.figure(figsize=(12,5))
#plt.title("Distribution of EXT_SOURCE_1")
#ax = sns.distplot(app_train["EXT_SOURCE_1"])

Analysis
Normalized score from external data source

### <a id="eda_app_train_ext_source_2">3.1.? EXT_SOURCE_2</a>

In [10]:
# grafico

Analysis - Tal vez los puedo juntar
Normalized score from external data source

### <a id="eda_app_train_ext_source_3">3.1.? EXT_SOURCE_3</a>

In [11]:
# Grafico

Analysis
Normalized score from external data source

### <a id="eda_app_train_days_birth">3.1.? DAYS_BIRTH</a>

In [12]:
'''
# NORMALIZE
from sklearn.preprocessing import MinMaxScaler
app_train_copy = app_train.copy()

scaler = MinMaxScaler()
# Full list of top ten features, discounting EXT_SOURCE_X becuase they are already normalizaed:
# ['DAYS_BIRTH', 'AMT_ANNUITY', 'AMT_CREDIT', 'DAYS_ID_PUBLISH', 'pcb_CNT_INSTALMENT_FUTURE', 'DAYS_REGISTRATION', 'DAYS_EMPLOYED']

# numerical = ['DAYS_BIRTH', 'AMT_ANNUITY', 'AMT_CREDIT', 'DAYS_ID_PUBLISH']
# 12 entries in 'AMT_ANNUITY' are NaN - I need to fix that first before Normalizing

# 'pcb_CNT_INSTALMENT_FUTURE' belongs to a different dataset

# numerical = ['DAYS_BIRTH', 'AMT_CREDIT', 'DAYS_ID_PUBLISH']

numerical = ['DAYS_BIRTH', 'AMT_CREDIT', 'DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'DAYS_EMPLOYED']


app_train_copy[numerical] = scaler.fit_transform(app_train_copy[numerical])

display(app_train_copy[numerical].head())

display(app_train_copy[numerical].isnull().sum().sum())


# SKEWED VALUES:
#   DAYS_EMPLOYED
'''

In [13]:
plt.figure(figsize=(12,5))
plt.title("Distribution of DAYS_BIRTH")
ax = sns.distplot(app_train["DAYS_BIRTH"])

### <a id="eda_app_train_amt_annuity">3.1.? AMT_ANNUITY</a>

In [14]:
'''
# ERROR: ValueError: cannot convert float NaN to integer

plt.figure(figsize=(12,5))
plt.title("Distribution of AMT_ANNUITY")
ax = sns.distplot(app_train["AMT_ANNUITY"])

plt.figure(figsize=(12,5))
plt.title("Distribution of AMT_ANNUITY")
ax = sns.distplot(app_train_copy["AMT_ANNUITY"])
'''

### <a id="eda_app_train_amt_credit">3.1.? AMT_CREDIT</a>

In [15]:
plt.figure(figsize=(12,5))
plt.title("Distribution of AMT_CREDIT")
ax = sns.distplot(app_train["AMT_CREDIT"])

### <a id="eda_app_train_days_id_publish">3.1.? DAYS_ID_PUBLISH</a>

In [16]:
plt.figure(figsize=(12,5))
plt.title("Distribution of DAYS_ID_PUBLISH")
ax = sns.distplot(app_train["DAYS_ID_PUBLISH"])

### <a id="eda_app_train_days_registration">3.1.? DAYS_REGISTRATION</a>

In [17]:
plt.figure(figsize=(12,5))
plt.title("Distribution of DAYS_REGISTRATION")
ax = sns.distplot(app_train["DAYS_REGISTRATION"])

### <a id="eda_app_train_days_employed">3.1.? DAYS_EMPLOYED</a>

In [18]:
plt.figure(figsize=(12,5))
plt.title("Distribution of DAYS_EMPLOYED")
ax = sns.distplot(app_train["DAYS_EMPLOYED"])

### <a id="eda_posc_bal_cnt_installment_future">3.?.? CNT_INSTALLMENT_FUTURE</a>

In [19]:
app_train["DAYS_EMPLOYED"].head(100)

## <a id="4.1">4.1 Feature Engineering - Create a Flat Dataset</a>

### <a id="4.1.1">4.1.1 Feature Engineering - Create a Flat Dataset - Previous Applications</a>

In [20]:
if debugging:
    display(app_train.head(3))
    display(app_test.head(3))

In [21]:
# Merge training and testing datasets - This will help in two ways:
# When handling categorical variables it will ensure both datasets end up with the same features
# When handling missing values, if we use the mean to fill in missing values, they will be more representative
app_train['is_train'] = 1
app_train['is_test'] = 0
app_test['is_train'] = 0
app_test['is_test'] = 1
print("\nJoining the training(app_train) and testing(app_test) dataset for pre-processing into pandas DataFrame 'data'.")

# data = pd.concat([app_train, app_test], axis=0, sort=False)
# ERROR: TypeError: concat() got an unexpected keyword argument 'sort'
data = pd.concat([app_train, app_test], axis=0)
# Substract 4 from the features count for the columns 'TARGET', 'SK_ID_CURR', 'is_train', 'is_test' for app_train
# And substract 3 for app_test, as it doesn't have a 'TARGET' column
print("app_train has {0:,} samples and {1} features.".format(app_train.shape[0], app_train.shape[1]-4))
print("app_test has {0:,} samples and {1} features.".format(app_test.shape[0], app_test.shape[1]-3))
print("data has {0:,} samples and {1} features BEFORE one-hot encoding.".format(data.shape[0], data.shape[1]-4))
assert(data.shape[0] == app_train.shape[0] + app_test.shape[0])
assert(data.shape[1] >= max(app_train.shape[1], app_test.shape[1]))

In [22]:
if debugging:
    display(data.head(3))
    display(data[data['SK_ID_CURR'] == 100001])

In [23]:
# Handle Categorical variables - Turn categorical variables into numerical features using the one-hot encoding scheme
# Support function for one-hot encoding
def _one_hot_encoding(data):
    # http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
    return pd.get_dummies(data)

# Handle categorical variables
print("\nPerforming one-hot encoding on {} dataset.".format('data'))
data = _one_hot_encoding(data)
# Substract 4 from the features count for the columns 'TARGET', 'SK_ID_CURR', 'is_train', 'is_test'
print("app has {0:,} samples and {1} features AFTER one-hot encoding.".format(data.shape[0], data.shape[1]-4))
posc_bal = _one_hot_encoding(posc_bal)
#bureau_bal = _one_hot_encoding(bureau_bal)
prev_app = _one_hot_encoding(prev_app)
inst_pay = _one_hot_encoding(inst_pay)
cc_bal = _one_hot_encoding(cc_bal)
bureau = _one_hot_encoding(bureau)

In [24]:
if debugging:
    display(data.head(3))

In [25]:
# Keep a copy of the application_train & application_test datasets without merging with the rest of the datasets
data_train_test = data.copy()

In [26]:
# Merge Point of Sale Cash Balance dataset
print("Merge 'Point of Sale Cash Balance' dataset.")
# Count the number of previous applications for a given 'SK_ID_CURR', and create a new feature
posc_bal_count = posc_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
posc_bal['POSC_BAL_COUNT'] = posc_bal['SK_ID_CURR'].map(posc_bal_count['SK_ID_PREV'])
# Remove the 'SK_ID_PREV' column from the dataset as it doesn't add value
posc_bal = posc_bal.drop(['SK_ID_PREV'], axis=1)

# Average values for all other features in previous applications
posc_bal_avg = posc_bal.groupby('SK_ID_CURR').mean()
posc_bal_avg.columns = ['pcb_' + col for col in posc_bal_avg.columns]
data = data.merge(right=posc_bal_avg.reset_index(), how='left', on='SK_ID_CURR')

In [27]:
'''
# Merge Bureau Balance dataset
print("Merge 'Bureau Balance' dataset.")
#'SK_ID_BUREAU'
# Count the number of previous applications for a given 'SK_ID_CURR', and create a new feature
bureau_bal_count = bureau_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
bureau_bal['bureau_bal_COUNT'] = bureau_bal['SK_ID_CURR'].map(bureau_bal_count['SK_ID_PREV'])
# Remove the 'SK_ID_PREV' column from the dataset as it doesn't add value
bureau_bal = bureau_bal.drop(['SK_ID_PREV'], axis=1)

# Average values for all other features in previous applications
bureau_bal_avg = bureau_bal.groupby('SK_ID_CURR').mean()
bureau_bal_avg.columns = ['posc_' + col for col in bureau_bal_avg.columns]
data_train = data_train.merge(right=bureau_bal_avg.reset_index(), how='left', on='SK_ID_CURR')
'''

In [28]:
if debugging:
    display(prev_app.shape)
    display(prev_app.head())
    display(prev_app[prev_app['SK_ID_CURR'] == 271877])
    display(data.shape)
    display(data.head())

In [29]:
# Merge Previous Applications dataset
print("Merge 'Previous Applications' dataset.")
# Count the number of previous applications for a given 'SK_ID_CURR'
prev_app_count = prev_app[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
prev_app['PREV_COUNT'] = prev_app['SK_ID_CURR'].map(prev_app_count['SK_ID_PREV'])
# Remove the 'SK_ID_PREV' column from the dataset as it doesn't add value
prev_app = prev_app.drop(['SK_ID_PREV'], axis=1)

# Average values for all other features in previous applications
prev_app_avg = prev_app.groupby('SK_ID_CURR').mean()
prev_app_avg.columns = ['pa_' + col for col in prev_app_avg.columns]
data = data.merge(right=prev_app_avg.reset_index(), how='left', on='SK_ID_CURR')

In [30]:
if debugging:
    display(prev_app_count.head())
    display(prev_app_avg.shape)
    display(prev_app_avg.head())
    #display(prev_app_avg[prev_app_avg['SK_ID_CURR'] == 271877].head())
    display(data.shape)
    display(data['pa_AMT_ANNUITY'].head())
    print("Total number of NaN in the data dataframe: {:,}".format(data.isnull().sum().sum()))

In [31]:
# Merge Installments Payments dataset
print("Merge 'Installments Payments' dataset.")
# Count the number of installments payments for a given 'SK_ID_CURR', and create a new feature
inst_pay_count = inst_pay[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
inst_pay['INST_PAY_COUNT'] = inst_pay['SK_ID_CURR'].map(inst_pay_count['SK_ID_PREV'])
# Remove the 'SK_ID_PREV' column from the dataset as it doesn't add value
inst_pay = inst_pay.drop(['SK_ID_PREV'], axis=1)

## Average values for all other features in previous applications
inst_pay_avg = inst_pay.groupby('SK_ID_CURR').mean()
inst_pay_avg.columns = ['ip_' + col for col in inst_pay_avg.columns]
data = data.merge(right=inst_pay_avg.reset_index(), how='left', on='SK_ID_CURR')

In [32]:
# Merge Credit Card Balance dataset
print("Merge 'Credit Card Balance' dataset.")
# Count the number of previous applications for a given 'SK_ID_CURR', and create a new feature
cc_bal_count = cc_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
cc_bal['CC_BAL_COUNT'] = cc_bal['SK_ID_CURR'].map(cc_bal_count['SK_ID_PREV'])
# Remove the 'SK_ID_PREV' column from the dataset as it doesn't add value
cc_bal = cc_bal.drop(['SK_ID_PREV'], axis=1)

## Average values for all other features in previous applications
cc_bal_avg = cc_bal.groupby('SK_ID_CURR').mean()
cc_bal_avg.columns = ['ccb_' + col for col in cc_bal_avg.columns]
data = data.merge(right=cc_bal_avg.reset_index(), how='left', on='SK_ID_CURR')

In [33]:
if debugging:
    display(bureau.head())
    display(bureau.shape)

In [34]:
# Merge Bureau dataset
print("Merge 'Bureau' dataset.")
# Count the number of credits registered in the bureau for a given 'SK_ID_CURR', and create a new feature
bureau_count = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
bureau['BUREAU_COUNT'] = bureau['SK_ID_CURR'].map(bureau_count['SK_ID_BUREAU'])
# Remove the 'SK_ID_BUREAU' column from the dataset as it doesn't add value
bureau = bureau.drop(['SK_ID_BUREAU'], axis=1)

## Average values for all other features in previous applications
bureau_avg = bureau.groupby('SK_ID_CURR').mean()
bureau_avg.columns = ['b_' + col for col in bureau_avg.columns]
data = data.merge(right=bureau_avg.reset_index(), how='left', on='SK_ID_CURR')

In [35]:
if debugging:
    display(bureau_avg.head())
    display(bureau_avg.shape)

In [36]:
# Transforming skewed continuous features
#skewed = ['DAYS_EMPLOYED']
#data[skewed] = data[skewed].apply(lambda x: np.log(x + 1))
# I need to handle negative numbers, if x = -1 then it will throw an error; log(0) = Inf

In [37]:
plt.figure(figsize=(12,5))
plt.title("Distribution of DAYS_EMPLOYED")
ax = sns.distplot(data["DAYS_EMPLOYED"])

In [38]:
# Normalizing numerical features
from sklearn.preprocessing import MinMaxScaler
#app_train_copy = app_train.copy()

scaler = MinMaxScaler()
# Full list of top ten features, discounting EXT_SOURCE_X becuase they are already normalizaed:
# ['DAYS_BIRTH', 'AMT_ANNUITY', 'AMT_CREDIT', 'DAYS_ID_PUBLISH', 'pcb_CNT_INSTALMENT_FUTURE', 'DAYS_REGISTRATION', 'DAYS_EMPLOYED']

# numerical = ['DAYS_BIRTH', 'AMT_ANNUITY', 'AMT_CREDIT', 'DAYS_ID_PUBLISH']
# 12 entries in 'AMT_ANNUITY' are NaN - I need to fix that first before Normalizing

# 'pcb_CNT_INSTALMENT_FUTURE' belongs to a different dataset

# numerical = ['DAYS_BIRTH', 'AMT_CREDIT', 'DAYS_ID_PUBLISH']

numerical = ['DAYS_BIRTH', 'AMT_CREDIT', 'DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'DAYS_EMPLOYED']
data[numerical] = scaler.fit_transform(data[numerical])

In [39]:
data_to_use = 'ALL' # 'ALL' or 'data_train_test'
if data_to_use == 'data_train_test':
    data = data_train_test.copy()

In [40]:
# Handle missing data
# https://pandas.pydata.org/pandas-docs/stable/missing_data.html#filling-with-a-pandasobject
# https://www.kaggle.com/dansbecker/handling-missing-values
# http://scikit-learn.org/dev/modules/generated/sklearn.impute.SimpleImputer.html
print("\nFilling NaN values in the dataset using pandas.fillna() using the column mean() value.")
print("Number of NaN values in the dataset BEFORE running pandas.fillna(): {:,}".format(data.isnull().sum().sum()))
data = data.fillna(data.mean())
nan_after = data.isnull().sum().sum()
print("Number of NaN values in the dataset AFTER running pandas.fillna(): {:,}".format(nan_after))
assert(nan_after == 0)

In [43]:
# Store preprocessed datasets into a file to start from this point when tunning hyperparameters
import gc

store_data = False
# Maybe I need to store the datasets in three parts: train, validation & test. So I don't run out of memory.
if store_data:
    # Clean variables that are no longer needed as I am running out of space when trying to save the preprocessed dataset
    posc_bal = posc_bal_count = posc_bal_avg = None
    bureau_bal = bureau_bal_count = bureau_bal_avg = None
    app_train = None
    prev_app = prev_app_count = prev_app_avg = None
    inst_pay = inst_pay_count = inst_pay_avg = None
    cc_bal = cc_bal_count = cc_bal_avg = None
    app_test = None
    bureau = bureau_count = bureau_avg = None
    data_train_test = None
    gc.collect()
    
    #data.to_csv('HCDR_DATA_ALL.csv')

In [ ]:
# Separate the data into the original test and training datasets
# Remove columns 'TARGET', 'SK_ID_CURR', 'is_train', 'is_test' as they are not features
print("\nSeparating the training and testing dataset after completing pre-processing.")
train = data[data['is_train'] == 1]
# Separate the 'target label' from the training dataset
target = train['TARGET']
train = train.drop(['TARGET', 'SK_ID_CURR', 'is_test', 'is_train'], axis=1)
test = data[data['is_test'] == 1]
# To be used when preparing the submission
test_id = test['SK_ID_CURR']
test = test.drop(['TARGET', 'SK_ID_CURR', 'is_test', 'is_train'], axis=1)
print("train has {0:,} samples and {1} features.".format(train.shape[0], train.shape[1]))
print("test has {0:,} samples and {1} features.".format(test.shape[0], test.shape[1]))

In [ ]:
# Split 'features' and 'target label' data into training and validation data using train_test_split
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
print("\nSplitting the training dataset into actual training and validation datasets")
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=42)
assert(train.shape[0] == X_train.shape[0] + X_val.shape[0])
assert(X_train.shape[1] == train.shape[1])
assert(X_val.shape[1] == train.shape[1])
assert(target.shape[0] == y_train.shape[0] + y_val.shape[0])
print("training dataset has {0:,} samples and {1} features.".format(X_train.shape[0], X_train.shape[1]))
print("validating dataset has {0:,} samples and {1} features.".format(X_val.shape[0], X_val.shape[1]))

In [ ]:
# Run GridSearchCV or fully train an estimator
run_mode = 'train_estimator_LGBM' # 'grid_search_RFR', 'grid_search_LGBM', 'train_estimator_LGBM', 'train_estimator_RFR'

In [ ]:
# Run GridSearchCV on LGBM
if run_mode == 'grid_search_LGBM':
    perc_samples = 0.15
    print("\nPreparing to run Hyperparameters tunning with GridSearchCV using {0:.2f}% of the training samples".format(perc_samples * 100))
    X_train_small = X_train[:int(perc_samples * X_train.shape[0])]
    y_train_small = y_train[:int(perc_samples * y_train.shape[0])]
    X_val_small = X_val[:int(perc_samples * X_val.shape[0])]
    y_val_small = y_val[:int(perc_samples * y_val.shape[0])]
    
    estimator = lgb.LGBMClassifier(
          objective='binary',
          metric='auc',
          num_iteration=5000, # num_boost_round=5000,
          verbose=1,
          silent=False,
          colsample_bytree=.8,
          subsample=.9,
          reg_alpha=.1,
          reg_lambda=.1,
          min_split_gain=.01,
          min_child_weight=1,
          # early_stopping_rounds=100
          # ValueError: For early stopping, at least one dataset and eval metric is required for evaluation
    )
    
    '''
    parameters = {
          'task': ['train'],
          'boosting_type': ['gbdt'],
          'objective': ['binary'],
          'metric': ['auc'],
          'learning_rate': [0.01],
          'num_leaves': [48],
          'num_iteration': [5000],
          'verbose': 0,
          'colsample_bytree': [.8],
          'subsample': [.9],
          'max_depth': [7],
          'reg_alpha': [.1],
          'reg_lambda': [.1],
          'min_split_gain': [.01],
          'min_child_weight': [1]
        }
    '''
    parameters = {
          'boosting_type': ['gbdt'], # 'dart'
          'num_leaves': [35, 48, 80],
          'min_data_in_leaf': [20], # [15, 20, 25],
          'learning_rate': [0.005],
          'max_depth': [7], # [6, 7, 8],
        }
    
    # Create a scorer to measure hyperparameters performance
    scorer = make_scorer(roc_auc_score)

    # Create GridSearchCV grid object
    grid_obj = GridSearchCV(estimator=estimator, 
                            param_grid=parameters, 
                            scoring=scorer)

    # Fit the GridSearchCV grid object with the reduced training dataset and find the best hyperparameters
    start = time()
    grid_fit = grid_obj.fit(X_train_small, y_train_small)
    end = time()
    grid_fit_time = (end - start) / 60 # Ellapsed time in minutes
    print("\nGridSearchCV estimator fit time: {0:.2f} minutes".format((end - start) / 60))

    print("\nPreparing to run Hyperparameters tunning with GridSearchCV using {0:.2f}% of the training samples".format(perc_samples * 100))
    print("\nParameters used for tunning: \n{}".format(parameters))
    # Get the best estimator
    best_est = grid_obj.best_estimator_
    print("\nBest Estimator: \n{}\n".format(best_est))

    # Get the best score
    best_score = grid_obj.best_score_
    print("\nBest Estimator Score: {}\n".format(best_score))

    # Get the best parameters
    best_params = grid_obj.best_params_
    print("\nBest Hyperparameters that yield the best score: \n{}\n".format(best_params))

    # Make predictions with unoptimized estimator on the validation set
    #pred_val = (estimator.fit(features_train_small, target_train_small)).predict(features_val_small)
    #print("\nUnoptimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(y_val_small, pred_val)))

    # Predict with the best estimator on the validation set
    best_pred_val = best_est.predict(X_val_small)
    print("\nOptimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(y_val_small, best_pred_val)))


In [ ]:
# Run GridSearchCV
if run_mode == 'grid_search_RFR':
    perc_samples = 0.15
    print("\nPreparing to run Hyperparameters tunning with GridSearchCV using {0:.2f}% of the training samples".format(perc_samples * 100))
    features_train_small = X_train[:int(perc_samples * X_train.shape[0])]
    target_train_small = y_train[:int(perc_samples * y_train.shape[0])]
    features_val_small = X_val[:int(perc_samples * X_val.shape[0])]
    target_val_small = y_val[:int(perc_samples * y_val.shape[0])]
    #features_test_small = features_test[:int(perc_samples * features_test.shape[0])]

    # Initialize the Estimator (Learner or Regression Model)
    estimator = RandomForestRegressor(n_jobs=-1,
                                      random_state=42,
                                      verbose=0)

    # Determine which Parameters to tune
    '''
    Tested so far:
    parameters = {
        'n_estimators': [9, 10, 11, 12, 13, 14, 15],
        'criterion': ['mse', 'mae'],
        'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7],
        'max_features': [0.01, 0.1, 0.25, 0.45, 0.5, 0.55, 0.6, 0.75],
        'min_samples_split': [2, 3, 4, 5],
        'warm_start': [False, True]
    }
    '''
    parameters = {
        'n_estimators': [130, 135, 145],
        'min_samples_leaf': [55, 62, 75],
        'max_features': [0.2], # [0.18, 0.2, 0.23]
        'min_samples_split': [2], # [2, 3]
    }

    # Create a scorer to measure hyperparameters performance
    scorer = make_scorer(roc_auc_score)

    # Create GridSearchCV grid object
    grid_obj = GridSearchCV(estimator=estimator, 
                            param_grid=parameters, 
                            scoring=scorer)

    # Fit the GridSearchCV grid object with the reduced training dataset and find the best hyperparameters
    start = time()
    grid_fit = grid_obj.fit(features_train_small, target_train_small)
    end = time()
    grid_fit_time = (end - start) / 60 # Ellapsed time in minutes
    print("\nGridSearchCV estimator fit time: {0:.2f} minutes".format((end - start) / 60))

    # Get the best estimator
    best_est = grid_obj.best_estimator_
    print("\nBest Estimator: \n{}\n".format(best_est))

    # Get the best score
    best_score = grid_obj.best_score_
    print("\nBest Estimator Score: {}\n".format(best_score))

    # Get the best parameters
    best_params = grid_obj.best_params_
    print("\nBest Hyperparameters that yield the best score: \n{}\n".format(best_params))

    # Make predictions with unoptimized estimator on the validation set
    pred_val = (estimator.fit(features_train_small, target_train_small)).predict(features_val_small)
    print("\nUnoptimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, pred_val)))

    # Predict with the best estimator on the validation set
    best_pred_val = best_est.predict(features_val_small)
    print("\nOptimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, best_pred_val)))

    # Predict with the best estimator on the testing set
    #pred_test = best_est.predict(features_test)

In [ ]:
# Train estimator LGBM
if run_mode == 'train_estimator_LGBM':
    lgb_train = lgb.Dataset(data=X_train, label=y_train)
    lgb_eval = lgb.Dataset(data=X_val, label=y_val)
    
    # https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772
    params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
              'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 5000, 'verbose': 0 ,
              'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
              'min_split_gain':.01, 'min_child_weight':1}
    
    '''
    # https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm/code
    params = {
        'n_estimators': 4000,
        'learning_rate': 0.03,
        'num_leaves': 30,
        'colsample_bytree': .8,
        'subsample': .9,
        'max_depth': 7,
        'reg_alpha': .1,
        'reg_lambda': .1,
        'min_split_gain': .01,
        'min_child_weight': 2,
        'silent': -1,
        'verbose': -1,
        'metric': 'auc',
    }
    '''
    
    start = time()
    estimator = lgb.train(params, lgb_train, valid_sets=lgb_eval, early_stopping_rounds=250, verbose_eval=100)
    end = time()
    #print("\nPreparing to train the following estimator: \n{}".format(estimator))
    print("\nEstimator fit time: {} seconds".format(int(round(end - start))))

    lgb.plot_importance(estimator, figsize=(12, 25), max_num_features=100);

In [ ]:
# Train estimator RandonForrestRegressor
if run_mode == 'train_estimator_RFR':
    # Initialize the Estimator (Learner or Regression Model) with the best hyperparameters
    # Alternative: n_estimators=135, max_features=0.2, min_samples_split=2, min_samples_leaf=62
    # Alternative2: criterion='mae', # default='mse', VERY SLOW
    estimator = RandomForestRegressor(n_estimators=125, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=2, # default=2
                                      min_samples_leaf=75, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=0) # default=0
    print("\nPreparing to train the following estimator: \n{}".format(estimator))

    # Fit the estimator with the training dataset
    start = time()
    estimator.fit(X_train, y_train)
    end = time()
    print("\nEstimator fit time: {} seconds".format(int(round(end - start))))

    # Predict with the validation dataset
    pred_val = estimator.predict(X_val)
    print("\nEstimator prediction score on Validation set: \t{}".format(roc_auc_score(y_val, pred_val)))
    
    # Determine the feature importance
    fi = pd.DataFrame()
    fi['feature'] = X_train.columns
    fi['importance'] = estimator.feature_importances_
    display(fi.sort_values(by=['importance'], ascending=False).head(10))

    # TODO: GRAPH THE FEATURE IMPORTANCE

In [ ]:
if 'train_estimator_' in run_mode:
    # Predict using the 'test' dataset for submission
    pred_test = estimator.predict(test)
    
    # Prepare prediction for submission
    print("\nPreparing prediction for submission.")
    submission = pd.DataFrame()
    submission['SK_ID_CURR'] = test_id
    # Replace any negative number with zero, required for https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm/code
    # pred_test[pred_test < 0] = 0
    submission['TARGET'] = pred_test
    submission.head()
    file_name = run_mode.split('train_estimator_')[1] + '.csv'
    submission.to_csv(file_name, index=False)